In [3]:
import os
import random
import pyperclip
from tkinter import *
from tkinter.ttk import *
from cryptography.fernet import Fernet

# ---------------- SECURITY SETUP ----------------
def generate_key():
    """Generate and save a secret key if not exists"""
    if not os.path.exists("key.key"):
        key = Fernet.generate_key()
        with open("key.key", "wb") as key_file:
            key_file.write(key)

def load_key():
    """Load the previously generated key"""
    return open("key.key", "rb").read()

generate_key()
key = load_key()
fernet = Fernet(key)

# ---------------- FILE SETUP ----------------
def checkExistence():
    """Ensure data file exists"""
    if not os.path.exists("info.txt"):
        with open("info.txt", 'w') as file:
            file.write("=== Encrypted Password Manager Data ===\n\n")

# ---------------- PASSWORD LOGIC ----------------
def generate_password():
    entry.delete(0, END)
    length = var1.get()

    lower = "abcdefghijklmnopqrstuvwxyz"
    upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    digits = "0123456789"
    symbols = "!@#$%^&*()"

    if var.get() == 1:  # Low
        chars = lower
    elif var.get() == 2:  # Medium
        chars = lower + upper + digits
    elif var.get() == 3:  # Strong
        chars = lower + upper + digits + symbols
    else:
        chars = lower

    password = ''.join(random.choice(chars) for _ in range(length))
    entry.insert(0, password)

def copy_password():
    random_password = entry.get()
    pyperclip.copy(random_password)

# ---------------- FILE OPERATIONS ----------------
def appendNew():
    """Save new encrypted password"""
    userName = entry1.get().strip()
    website = entry2.get().strip()
    Random_password = entry.get().strip()

    if not userName or not website or not Random_password:
        print("All fields are required!")
        return

    # Combine data and encrypt
    data = f"Username: {userName}\nWebsite: {website}\nPassword: {Random_password}\n"
    encrypted_data = fernet.encrypt(data.encode())

    with open("info.txt", 'ab') as file:  # binary mode
        file.write(encrypted_data + b"\n-----\n")

    entry1.delete(0, END)
    entry2.delete(0, END)
    entry.delete(0, END)
    print("Password saved securely!")

def readPasswords():
    """Decrypt and display saved passwords"""
    if not os.path.exists("info.txt"):
        return

    with open('info.txt', 'rb') as file:
        content = file.read().split(b"\n-----\n")

    decrypted_data = ""
    for item in content:
        if item.strip():
            try:
                decrypted_data += fernet.decrypt(item).decode() + "\n---------------------------------\n"
            except Exception:
                continue

    top = Toplevel(root)
    top.title("Saved Passwords")
    text_box = Text(top, wrap=WORD, width=60, height=20)
    text_box.insert(END, decrypted_data if decrypted_data else "No valid data found.")
    text_box.config(state=DISABLED)
    text_box.pack(padx=10, pady=10)

# ---------------- GUI SETUP ----------------
checkExistence()
root = Tk()
root.title("Secure Password Manager")

var = IntVar(value=2)  # Default medium
var1 = IntVar(value=12)

Label(root, text="Password Length:").grid(row=0, column=0, sticky='W')
combo = Combobox(root, textvariable=var1, width=5)
combo['values'] = tuple(range(8, 33))
combo.grid(row=0, column=1)

Radiobutton(root, text="Low", variable=var, value=1).grid(row=0, column=2)
Radiobutton(root, text="Medium", variable=var, value=2).grid(row=0, column=3)
Radiobutton(root, text="Strong", variable=var, value=3).grid(row=0, column=4)

Button(root, text="Generate", command=generate_password).grid(row=0, column=5)
Button(root, text="Copy", command=copy_password).grid(row=0, column=6)

Label(root, text="Username:").grid(row=1, column=0, sticky='W')
entry1 = Entry(root, width=25)
entry1.grid(row=1, column=1, columnspan=3)

Label(root, text="Website:").grid(row=2, column=0, sticky='W')
entry2 = Entry(root, width=25)
entry2.grid(row=2, column=1, columnspan=3)

Label(root, text="Password:").grid(row=3, column=0, sticky='W')
entry = Entry(root, width=25)
entry.grid(row=3, column=1, columnspan=3)

Button(root, text="Save", command=appendNew).grid(row=3, column=5)
Button(root, text="Show All", command=readPasswords).grid(row=3, column=6)

root.mainloop()
